# Introduction 

## Data wrangling and transformation

Often, datasets need to go through a series of data wrangling and transformation steps before they are ready for analysis or visualisation tasks. This lab will demonstrate several data wrangling and transformation operations for raster, vector, and tabular data. 

We will start with a subset of the AgriFieldNet Competition Dataset <a href="https://mlhub.earth/data/ref_agrifieldnet_competition_v1" target="_blank">(Radiant Earth Foundation and IDinsight, 2022)</a> which has been published to encourage people to develop machine learning models that classify a field's crop type from satellite images. This dataset consists of a series of directories with each directory corresponding to a 256 x 256 pixel image footprint. Inside each directory are the following files:

* 12 GeoTIFF files corresponding to spectral reflectance in different wavelengths from Sentinel-2 data. 
* 1 GeoTIFF file with non-zero pixels corresponding to a crop type label. 
* 1 GeoTIFF file with non-zero pixels corresponding to a field id. 
* 1 JSON metadata file. 

This data is subset from a larger dataset covering agricultural fields in four Indian states of Odisha, Uttar Pradesh, Bihar, and Rajasthan. The field boundaries and crop type labels were captured by data collectors from IDinsights Data on Demand team and the satellite image preparation was undertaken by the Radiant Earth Foundation. 

Our task is to combine all the raster data in separate directories into a tabular dataset that can be used for machine learning tasks to predict a field's crop type. Specifically, we will  transform a disparate collection of GeoTiff files spread across many directories and storing different information into a tabular dataset with columns for each field id, crop type, and average spectral reflectance values in each field for many wavelenghts. We will also store geometry data representing the location of each field in a geometry column. 

Through completing this task you will use programming skills from week 1, build on your understanding of directories and file systems from week 2, and tools for data cleaning covered in week 3. You will build on these skills by learning a range of common data wrangling and transformation operations which you will be able to apply to a range of datasets to wrangle them into a structure suitable for analysis and visualisation. You will also learn how to use Python to create an automated data transformation routine consisting of several operations to automate the processing of a collection of datasets into a format ready for analysis.


### What is data wrangling?

<a href="https://r4ds.had.co.nz/wrangle-intro.html" target="_blank">Wickham and Grolemund (2017)</a> and <a href="https://wesmckinney.com/book/" target="_blank">McKinney (2022)</a> state that data wrangling consists of data import, data cleaning, and data transformation. 

#### Data import

Data import was covered in week 2 with examples of how to read tabular, vector, and raster data into Python programs. 

#### Data cleaning

Data cleaning was covered in week 3 as part of the exploratory data analysis with examples of how to handle outliers and missing data. 

#### Data transformation

<a href="https://wesmckinney.com/book/" target="_blank">McKinney (2022)</a> define data transformation as the application of mathematical or statistical operations to data to generate new datasets. Data transformation can also include operations that reshape datasets or combine two or more datasets.

As we're working with spatial and non-spatial data we can categorise data transformation operations as attribute operations, spatial operations, geometry operations, and raster-vector  operations (<a href="https://geocompr.robinlovelace.net/index.html" target="_blank">Lovelace et al. (2022)</a>).

**Attribute operations** are applied to non-spatial (attribute data). This could be a tabular dataset without any spatial information, the attribute table of a vector dataset, or the pixel values of a raster dataset. Common attribute operations include:

* Selecting columns from a table based on a condition. 
* Selecting (subsetting) pixels from a raster based on a condition.
* Filtering rows from a table based on a condition. 
* Creating a new column of values using a function applied to existing data.
* Computing summary statistics of columns in a table or of pixel values in a raster.
* Joining datasets based on matching values in columns (keys).

**Spatial operations** transform data using the data's geographic information including shape and location. Vector spatial operations include:

* Spatial subsetting by selecting data points based on a geographic condition (e.g. selecting all fields in Western Australia).
* Spatial joins where datasets are combined based on their relationship in space. 
* Spatial aggregation where summaries are produced for regions (e.g. the average crop yield for all fields in a region).

Spatial operations on raster data are based on map algebra concepts and include:

* Local operations which are applied on a pixel by pixel basis (e.g. converting a raster of temperature values in °F to °C).
* Focal operations which summarise or transform a raster value using the values of neihbouring pixels (e.g. computing the average value within a 3 x 3 pixel moving window).
* Zonal operations which summarise or transform raster values using values inside an irregular shaped zone.
* Global operations which summarise the entire raster (e.g. computing the minimum value in the raster dataset). 

**Geometry operations** transform a dataset's geographic information. Common geometry operations for vector data include:

* Simplification of shapes.
* Computing the centroid of polygons.
* Clipping (subsetting) of geometries based on their intersection or relationship with another geometry. 

and geometry operations on raster data typically involve changing the spatial resolution and include:

* Aggregation or dissagregation.
* Resampling

**Raster-vector operations** involve both raster and vector datasets and include:

* Cropping or masking raster data using a vector geometry.
* Extracting raster values that intersect with a vector geometry.
* Rasterisation where a vector dataset is transformed to a raster layer.
* Vectorisation where a raster dataset is transformed to a vector layer.

### Feature engineering

Feature engineering is part of a machine learning workflow which involves preparing and preprocessing datasets ready for machine learning model training and evaluation. Feature engineering is one example where data wrangling operations are applied. This lab can be considered a feature engineering task where a range of raster satellite image datasets are transformed to a vector-tabular dataset which can be used to train and test a machine learning model. Often, datasets for machine learning computer vision tasks (e.g. see the datasets on Radiant Earth's <a href="https://mlhub.earth/" target="_blank">MLHub</a>) are provided with data samples for model development spread across many sub-directories. Prior to model training you need to extract the data from these directories and assemble it in a way that it can be passed into a model. You can use this lab as a starter template for these kind of feature engineering tasks.  

## Setup

### Run the labs

You can run the labs locally on your machine or you can use cloud environments provided by Google Colab or Binderhub. **If you're working with Google Colab and Binderhub be aware that your sessions are temporary and you'll need to take care to save, backup, and download your work.**

<a href="https://colab.research.google.com/github/data-analysis-3300-3003/colab/blob/main/lab-4.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<a href="https://mybinder.org/v2/gh/binder-3300-3003/binder/HEAD" target="_blank">
  <img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/>
</a>

### Download data

If you need to download the data for this lab, run the following code snippet. 

In [ ]:
import os

if "week-4" not in os.listdir(os.getcwd()):
    os.system('wget "https://github.com/data-analysis-3300-3003/data/raw/main/data/week-4.zip"')
    os.system('unzip "week-4.zip"')

### Working in Colab

If you're working in Google Colab, you'll need to run the following code snippet to install the required packages that don't come with the colab environment.

In [ ]:
!pip install geopandas
!pip install pyarrow
!pip install mapclassify
!pip install rasterio
!pip install libpysal
!pip install esda
!pip install splot

### Import modules

In [ ]:
# Import modules
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import plotly.io as pio
import shapely.geometry
import pprint

from rasterio import features

pio.renderers.default = "colab"

## Data storage

In week 2 we showed that files are organised within a hierarchy of directories and sub-directories (or folders) in a computer system. Each sample (256 x 256 pixel image) is stored in its own sub-directory. We can explore the structure of these directories and how files are arranged within it. 

First, let's list the first level of sub-directories within the `week-4` folder. Each of these sub-directories corresponds to a 256 x 256 pixel image.

To recap, we can use functions provided by the `os` package to help us explore directories. The `os.path.join()` function takes a sequence of string data representing directory names or file names and combines them into a path. The `os.listdir()` function lists all files or sub-directories in a directory pointed to by a path. 

In [ ]:
# Load the image and labels data
data_path = os.path.join(os.getcwd(), "week-4", "images")

image_dirs = os.listdir(data_path)
for i in image_dirs:
    if i != ".DS_Store":
        print(i)
    
if ".DS_Store" in image_dirs:
    image_dirs.remove(".DS_Store")

The `os.listdir()` function lists files or directories at the first level of the directory passed into the function as an argument. However, often we have nested directory structures consisting of a hierarchy of sub-directories and files. The `os.walk()` can be used to traverse a directory tree. We can use the `os.walk()` function to fully reveal how the files are arranged within a sub-directory. 

We can see that within each sub-directory corresponding to a 256 x 256 pixel image footprint there are the following files and sub-directories:

* Files with the names `B*.tif` which are Sentinel-2 images for a particular waveband. For example, B02.tif is Sentinel-2 reflectance in the blue visible wavelength. These files will be used to generate predictor variables in a subsequent machine learning task to predict crop type.
* A `field` sub-directory which contains a `field_ids.tif` file. This file stores field ids as pixel values. 
* A `label` sub-directory which contains a `raster_labels.tif` file. This file stores a numeric indicator of crop type as pixel values. These files store target labels used to train and test a machine learning model that predicts crop type from spectral reflectance data.

In [ ]:
for root, dirs, files in os.walk(os.path.join(data_path, image_dirs[1]), topdown=True):
    print(root)
    for f in files:
        print(f"    {f}")

### Data visualisation

Let's quickly visualise some of this data to get an idea of its structure. First, let's visualise the Sentinel-2 satellite image data starting with the image storing reflectance in the green visible portion of the electromagnetic spectrum. 

We can see that the image shape is 256 x 256 pixels. 

In [ ]:
# path to the green band GeoTIFF file
s2_green_path = os.path.join(os.getcwd(), "week-4", "images", image_dirs[1], "B03.tif")

# open the green band GeoTIFF file and read its image data
with rasterio.open(s2_green_path) as src:
    green_band = src.read(1)
    print(f"the shape of the image is {green_band.shape}")

# plot the green band
px.imshow(green_band, color_continuous_scale="Greens")

Let's also create a true colour composite image using the red, green, and blue band images.

In [ ]:
# path to the red band GeoTIFF file
s2_red_path = os.path.join(os.getcwd(), "week-4", "images", image_dirs[1], "B04.tif")

# open the red band GeoTIFF file and read its image data
with rasterio.open(s2_red_path) as src:
    red_band = src.read()

# path to the green band GeoTIFF file
s2_green_path = os.path.join(os.getcwd(), "week-4", "images", image_dirs[1], "B03.tif")

# open the green band GeoTIFF file and read its image data
with rasterio.open(s2_green_path) as src:
    green_band = src.read()
    
# path to the blue band GeoTIFF file
s2_blue_path = os.path.join(os.getcwd(), "week-4", "images", image_dirs[1], "B02.tif")

# open the blue band GeoTIFF file and read its image data
with rasterio.open(s2_blue_path) as src:
    blue_band = src.read()
    
# make RGB image
rgb = np.concatenate((red_band, green_band, blue_band), axis=0)

# plot the rgb image
px.imshow(np.moveaxis(rgb, 0, 2), contrast_rescaling="minmax")

Now, let's explore the field id images. We can see that the image contains a few fields with ids assigned to them. Hover over each field and you can see its numeric id value. However, we can also see that a large portion of the image is covered by pixels with the value 0. These locations are not labelled fields and we will need to drop them from our dataset. 

In [ ]:
# path to the GeoTIFF file
field_id_path = os.path.join(os.getcwd(), "week-4", "images", image_dirs[1], "field/field_ids.tif")

# open the GeoTIFF file and read its image data
with rasterio.open(field_id_path) as src:
    field_id_band = src.read(1)

# plot the field id band
px.imshow(field_id_band)

Finally, we can look at our labels image. Each field's pixels are assigned a numeric value that corresponds to a crop type. Based on the dataset's documentation the below is the mapping between numeric values and crop types in the labels dataset. 

* 1 - Wheat
* 2 - Mustard
* 3 - Lentil
* 4 - No crop/Fallow
* 5 - Green pea
* 6 - Sugarcane
* 8 - Garlic
* 9 - Maize
* 13 - Gram
* 14 - Coriander
* 15 - Potato
* 16 - Bersem
* 36 - Rice

In [ ]:
# path to the GeoTIFF file
label_path = os.path.join(os.getcwd(), "week-4", "images", image_dirs[1], "label/raster_labels.tif")

# open the GeoTIFF file and read its metadata and image data
with rasterio.open(label_path) as src:
    meta = src.meta
    label_band = src.read(1)

# Plot the crop label band
px.imshow(label_band)

## Raster data processing

### NumPy refresher

In week 2 we introduced NumPy `ndarray` objects for storing multidimensional (or N-dimensional) arrays which consist of a grid of elements of the same data type. `ndarray` objects are a logical data structure for representing and manipulating raster and image data in Python programs.

The dimensions of a NumPy `ndarray` are called axes. A single band raster layer would have two axes, rows would be arranged along the 0th axis and columns along the 1st axis. The shape of an `ndarray` refers to the number of elements along each axis. 

Let's quickly revise these concepts by working with a raster layer with 3 rows and 3 columns. 

In [ ]:
demo_raster = np.array(
    [[1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]])
print(demo_raster)

This `ndarray` object should have 2 axes corresponding to rows (0 axis) and columns (1 axis) with 3 elements along each axis. 

In [ ]:
print(f"the shape of demo_raster is {demo_raster.shape}")
print(f"the number of elements on the 0 axis (rows) is {demo_raster.shape[0]}")
print(f"the number of elements on the 1 axis (columns) is {demo_raster.shape[1]}")

### Subsetting NumPy `ndarray`s

This is a form of subsetting opertation where you select values from a NumPy `ndarray` object based on their index locations. These operations are generally referred to as indexing and slicing when working with NumPy `ndarray` objects. <a href="https://geocompr.robinlovelace.net/index.html" target="_blank">Lovelace et al. (2022)</a> refer to these operations on raster data as *row-column subsetting*.

We can extract a value from a NumPy `ndarray` based on its index location. For example, the first element of a 2-Dimensional `ndarray` is at location `[0, 0]` (i.e. the 0th row and 0th column). 

In [ ]:
first_element = demo_raster[0, 0]
print(first_element)

We can use the `:` symbol to specify slices of a NumPy `ndarray` to subset. For example, the following are three different ways of slicing the first two rows.

Note that the slice is not inclusive of the index location after the `:` symbol. So, `demo_raster[0:2, ]` would select the first two rows of `demo_raster` - row 0 and row 1 (remember Python indexes from 0).

In [ ]:
two_rows_1 = demo_raster[0:2, ]
print(two_rows_1)

two_rows_2 = demo_raster[0:2]
print(two_rows_2)

two_rows_3 = demo_raster[:2]
print(two_rows_3)

We can use multiple slices for different axes. For example, if we wanted to subset values from a selection of rows and columns. 

In [ ]:
two_rows_cols = demo_raster[:2, 1:]
print(two_rows_cols)

### Band stacking 

A common data wrangling and combination operation when working with raster data is band stacking. This is the process of taking two or more single band rasters and stacking them to create a multiband raster. 

When using NumPy `ndarray` objects to handle raster data, this process could involve stacking two `ndarray` objects with a shape of `(256, 256)` to create a new `ndarray` object with the shape `(2, 256, 256)`. Here, axis 0 has a length of two which indicates that we've stacked two `ndarray` objects with the shape `(256, 256)`.

We can use the NumPy `stack()` and `concatenate()` functions to combine a sequence of `ndarray`s along an axis. 

We can write a short code snippet to start our automated data transformation routine that will loop over image directories and Sentinel-2 bands, read the data stored in GeoTIFF files corresponding to Sentinel-2 reflectance values into an `ndarray`, and then use NumPy's `stack()` function to stack a list of `ndarray`s representing a multiband raster structure. Let's break this down step by step:

We loop over our list of directories (named in the list `image_dirs`) and inside each directory are a series of `B*.tif` files storing Sentinel-2 reflectance data. The outer loop over directories is: 

```python
for i in image_dirs:
    ## When inside this loop i takes on a directory name listed in image_dirs
    print(f"stacking Sentinel-2 bands in {i}")
```

Next, we create an empty list which we'll use to store `ndarray` objects of data read from the `B*.tif` GeoTIFF files. This is defined as `bands = []`, the `[]` creates an empty list:

```python
for i in image_dirs:
    print(f"stacking Sentinel-2 bands in {i}")
    # a list to store numpy ndarray objects of Sentinel-2 reflectance data for each band
    bands = []
```

Now, for each iteration of `i`, which corresponds to a directory, we iterate over a the `B*.tif` files inside `i` and read the data stored in each file into an `ndarray` and append the `ndarray` to the `bands` list using the `bands.append(src.read(1))` command. 

```python
for i in image_dirs:
    print(f"stacking Sentinel-2 bands in {i}")
    # a list to store numpy ndarray objects of Sentinel-2 reflectance data for each band
    bands = []
    
    # loop over each band, read in the data from the corresponding GeoTIFF file into an ndarray
    for b in s2_bands:
        band_path = os.path.join(os.getcwd(), data_path, i, b + ".tif")
        with rasterio.open(band_path) as src:
            # append the ndarray storing the Sentinel-2 reflectance data for a band to a list
            bands.append(src.read(1))
```

You will notice the use of the context manager denoted by the `with` block to read data from GeoTIFF files into a NumPy `ndarray`. This was covered in week 2, but let's revise this quickly: 

```python
with rasterio.open(band_path) as src:
```

Creates a file object referenced by `src` which points to the file at the path referenced by `band_path` (here a GeoTIFF file). The use of the `with` block as a context manager takes care of closing the connection `src` when we have finished reading data from the file. The file object `src` has a `read()` method which can be used to read data from the file it connects to into our Python program. 

```python
src.read(1)
```

For each `B*.tif` file in a directory, we read it's data into our program as: 

```python
for b in b2_bands:
    band_path = os.path.join(os.getcwd(), data_path, i, b + ".tif")
    with rasterio.open(band_path) as src:
        # append the ndarray storing the Sentinel-2 reflectance data for a band to a list
        bands.append(src.read(1))
```

Once we have finished looping over the bands in the directory `i`, we can stack the `ndarray` objects in the list `bands` to form a NumPy `ndarray` representation of a multiband raster. We do this by passing the list of `ndarray` objects into the NumPy `stack()` function.

```python
# loop over image directories
for i in image_dirs:
    print(f"stacking Sentinel-2 bands in {i}")
    # a list to store numpy ndarray objects of Sentinel-2 reflectance data for each band
    bands = []
    
    # loop over each band, read in the data from the corresponding GeoTIFF file into an ndarray
    for b in s2_bands:
        band_path = os.path.join(os.getcwd(), data_path, i, b + ".tif")
        with rasterio.open(band_path) as src:
            # append the ndarray storing the Sentinel-2 reflectance data for a band to a list
            bands.append(src.read(1))
    
    # stack all bands in the list to create a multiband raster
    bands = np.stack(bands)
```

Finally, before we more onto the next `i` (i.e. the next directory in the list `image_dirs`) and repeat this process, we append our `ndarray` object of stacked Sentinel-2 bands to the list `stacked_bands`. When we have finished looping over all directories in `image_dirs` the list `stacked_bands` will be a collection of `ndarray` objects that we can refer to later in our program.  


In [ ]:
# stacking bands 

# Sentinel-2 band names 
s2_bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

# a list to store numpy ndarray objects representing multiband rasters of Sentinel-2 reflectance data
stacked_bands = []

# loop over image directories
for i in image_dirs:
    print(f"stacking Sentinel-2 bands in {i}")
    # a list to store numpy ndarray objects of Sentinel-2 reflectance data for each band
    bands = []
    
    # loop over each band, read in the data from the corresponding GeoTIFF file into an ndarray
    for b in s2_bands:
        band_path = os.path.join(os.getcwd(), data_path, i, b + ".tif")
        with rasterio.open(band_path) as src:
            # append the ndarray storing the Sentinel-2 reflectance data for a band to a list
            bands.append(src.read(1))
    
    # stack all bands in the list to create a multiband raster
    bands = np.stack(bands)
    
    # append the multiband raster to the list storing multiband rasters for data in each directory
    stacked_bands.append(bands)


Let's inspect the output of the band stacking workflow. The list `stacked_bands` should be a list of `ndarray` objects with rank 3. Each `ndarray` object should have three axes (bands, rows, columns).

In [ ]:
idx = 0
for i in stacked_bands:
    print(f"the shape of the ndarray at position {idx} in stacked_bands is {i.shape} with rank {len(i.shape)}")
    idx += 1

### Map algebra

Following <a href="https://geocompr.robinlovelace.net/index.html" target="_blank">Lovelace et al. (2022)</a>, we refer to map algebra as operations that transform raster pixel values via statistical or mathematical operations which can involve combining pixel values from different raster layers or using neighbouring raster values. 

Local map algebra operations operate on a pixel by pixel basis; the mathematical operation is applied independently to each pixel withour reference to neighbouring pixel values. For example, addition, subtraction, multiplication, and logical operations can all be applied on a pixel by pixel basis. 

A commonly used local operation when working with remote sensing data is computing spectral indices. Spectral indices are pixel by pixel mathematical combinations of spectral reflectance in different wavelenghts that are used to emphasise and reveal vegetation or land surface conditions. The normalised difference vegetation index (NDVI) is used for tracking vegetation condition and representing the greenness of vegetation in a remote sensing image. 

As we're processing these remote sensing images to generate characteristics of fields that can be used to predict crop type, we will also compute each field's NDVI value as it could contain useful information to discriminate between crop types. **You will learn more about the theory and use of spectral and vegetation indices for remote sensing image analysis in GEOG 3301**. 

The NDVI is computed as:

$NDVI = \frac{NIR-red}{NIR+red}$

Thus, the NDVI is computed via division, subtraction, and addition operations computed on a pixel by pixel basis using raster data corresponding to red and near infrared reflectance. 

NumPy provides tools for fast mathematical operations on `ndarray` objects. If `ndarray` objects have the same shape, a mathematical combination of two or more `ndarray` objects will be computed on an element by element (i.e. pixel by pixel) basis without needing to write for loops to iterate over `ndarray` elements. This feature of NumPy is called *vectorisation* and makes NumPy a useful tool for processing and analysing large amounts of image data.

For example, if we wanted to add two NumPy `ndarray` objects together on a pixelwise basis we could do this using for loops in Python:

In [ ]:
# slow array addition using for loops
a = np.array(
    [[1, 2],
     [3, 4]])

b = np.array(
    [[1, 2],
     [3, 4]])   

result = np.zeros((2, 2))

for r in range(0, 2):
    print(f"processing row {r}")
    for c in range(0, 2):
        print(f"processing column {c}")
        
        result[r, c] = a[r, c] + b[r, c]

print(result)     

However, this approach will be slow if we are working with large raster datasets in NumPy `ndarray`s. It is also quite verbose, we need to write two for loops just to perform element-wise addition of two arrays. These element-wise operations can be computed in isolation. In other words adding the elements in pixel location `0, 0` does not depend on the result of adding elements in pixel location `0, 1`. This means that element-wise operations can be performed in parallel, which is termed vectorised computation in NumPy, and you can use NumPy's element-wise operations to perform mathematical operations on `ndarray`s in parallel. 

This has two advantages: speed (particularly when working with large or many images) and cleaner code. The NumPy element-wise approach to adding the `ndarray`s `a` and `b` above is:

In [ ]:
# vectorised addition using for loops
a = np.array(
    [[1, 2],
     [3, 4]])

b = np.array(
    [[1, 2],
     [3, 4]])  

result = a + b

print(result)

Now we're ready to use NumPy's element wise operations to compute the NDVI. First, let's do this for a single image before extending the routine to automate the generation of NDVI bands for all images. We'll get the first `ndarray` object from `stacked_bands`, a list of rank 3 (bands, rows, cols) `ndarray` objects with each band storing reflectance in different wavelengths. 

Red reflectance is stored in band 4 of Sentinel-2 images. Thus, red reflectance would be located at index position 3 (the fourth element as we start indexing at 0) on axis 0. 

Near infrared reflectance is stored in band 8 of Sentinel-2 images. By the same logic near infrared reflectance is located at index position 7 on axis 0. 

In [ ]:
# compute NDVI
image_1 = stacked_bands[0]
print(f"the shape of the first ndarray (image_1) in stacked bands is {image_1.shape}")

# get the red band
red = image_1[3, :, :].astype("float64")

# get the nir band
nir = image_1[7, :, :].astype("float64")

# compute the ndvi
ndvi = (nir-red)/(nir+red)

print(f"the shape of the ndvi image is {ndvi.shape}")

NDVI values fall between -1 and 1 with values closer to 1 indicating greener vegetation and values less than 0 indicating an absence of vegetation. Let's visualise the NDVI band and check the values fall within this range. 

In [ ]:
# visualise the ndvi image
px.imshow(ndvi, color_continuous_scale="viridis", contrast_rescaling="minmax")

You will have noticed that prior to computing the NDVI values we converted the red and near infrared `ndarray` objects to `float64` type. This is because NDVI values represent variation in vegetation conditions using numbers with digits before and after the decimal point (a NDVI value of 0.8 would indicate green vegetation and a NDVI value of -0.2 would indicate an absence of green vegetation and possibly water cover). Thus, it is more appropriate to use floating point numbers to store NDVI values.

If you check the data type (`dtype`) of the values of the multiband `ndarray` objects storing reflectance data you will see that they are of type `uint8`. This means they are unsigned integers - they cannot represent negative values. However, NDVI values can be negative. Therefore, we should convert these values to a data type that can store negative and positive (i.e. signed) numbers. 

We use the NumPy method `astype()` to cast an `ndarray` to a new `dtype`. You can read more about NumPy data types <a href="https://wesmckinney.com/book/numpy-basics.html#numpy_dtypes" target="_blank">here</a>.

In [ ]:
print(f"The dtype of the red band is {image_1[3, :, :].dtype}")

Now we have demonstrated how to compute NDVI values from NumPy `ndarray` objects, we can edit our existing routine that created rank 3 `ndarray` objects stacking by raster layers to also include an NDVI band. This will create a routine that not only automates the stacking of multiple GeoTIFF files in different directories but also the computation of NDVI bands for each image. 

In [ ]:
# stacking bands 

# Sentinel-2 band names 
s2_bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

# a list to store numpy ndarray objects representing multiband rasters of Sentinel-2 reflectance data
stacked_bands = []

# loop over image directories
for i in image_dirs:
    print(f"stacking Sentinel-2 bands in {i}")
    # a list to store numpy ndarray objects of Sentinel-2 reflectance data for each band
    bands = []
    
    # loop over each band, read in the data from the corresponding GeoTIFF file into an ndarray
    for b in s2_bands:
        band_path = os.path.join(os.getcwd(), data_path, i, b + ".tif")
        with rasterio.open(band_path) as src:
            # append the ndarray storing the Sentinel-2 reflectance data for a band to a list
            bands.append(src.read(1))
    
    # stack all bands in the list to create a multiband raster
    bands = np.stack(bands)
    
    # make NDVI band
    red = bands[3,:,:].astype("float64")
    nir = bands[7,:,:].astype("float64")
    ndvi = (nir-red)/(nir+red)
    ndvi = np.expand_dims(ndvi, axis=0) # add a bands axis
    bands = np.concatenate((bands, ndvi), axis=0) # stack the ndvi band
    
    # append the multiband raster to the list storing multiband rasters for data in each directory
    stacked_bands.append(bands)

Let's visualise the NDVI band in the first `ndarray` in `stacked_bands` to check it looks sensible. Also, note how we can use the index value `-1` for the last element along an axis. The NDVI band was stacked at the end of the 0 axis of the `ndarray` so the NDVI raster is the last slice of the `ndarray` on the 0 axis. 

In [ ]:
check_raster = stacked_bands[0]
print(f"the shape of the first ndarray in stacked_bands is {check_raster.shape}")

ndvi = check_raster[-1, :,  :]

# visualise the ndvi image
px.imshow(ndvi, color_continuous_scale="viridis", contrast_rescaling="minmax")

### Labels and field id bands

So, far we have built a routine that automates the processing of Sentinel-2 images using a series of operations applied to raster data. These are our predictor variables of crop type in a dataset that can be used to train a machine learning model that classifies crop type. Now we need to get the crop type labels that our model will learn to predict (classify) from the Sentinel-2 remote sensing data. 

We can extend our existing routine that iterates over directories listed in `image_dirs` to do this. After we generate and stack the NDVI band we can read in the field id and crop type labels data and append them to the `ndarray` object too. 

Look for the following code in the routine below to see how this is done: 

```python
### HERE WE ARE STACKING THE FIELD ID BAND ###
field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
with rasterio.open(field_id_path) as src:
    field_ids = src.read().astype("float64")
    field_ids[field_ids == 0] = np.nan
    bands = np.concatenate((bands, field_ids), axis=0)

### HERE WE ARE STACKING THE CROP TYPE LABELS BAND ###
labels_path = os.path.join(os.getcwd(), data_path, i, "label/raster_labels.tif")
with rasterio.open(labels_path) as src:
    bands = np.concatenate((bands, src.read()), axis=0)
```

A few things to note:

* the GeoTIFF files storing the field ids (`field_ids.tif`) and crop type labels (`raster_labels.tif`) are stored in sub-directories within each directory referenced by `i`. This means we need to create a path that points to these files within their sub-directories.
* pixels in the `field_ids.tif` raster with a value of 0 do not correspond to crop type labels. These pixels are not of interest to us here so we can set them to `np.nan`. Later we will drop all `np.nan` values so our dataset only reflects locations with crop type labels. 

In [ ]:
# stacking bands 

# Sentinel-2 band names 
s2_bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

# a list to store numpy ndarray objects representing multiband rasters of Sentinel-2 reflectance data
stacked_bands = []

# loop over image directories
for i in image_dirs:
    print(f"stacking Sentinel-2 bands in {i}")
    # a list to store numpy ndarray objects of Sentinel-2 reflectance data for each band
    bands = []
    
    # loop over each band, read in the data from the corresponding GeoTIFF file into an ndarray
    for b in s2_bands:
        band_path = os.path.join(os.getcwd(), data_path, i, b + ".tif")
        with rasterio.open(band_path) as src:
            # append the ndarray storing the Sentinel-2 reflectance data for a band to a list
            bands.append(src.read(1))
    
    # stack all bands in the list to create a multiband raster
    bands = np.stack(bands)
    
    # make NDVI band
    red = bands[3,:,:].astype("float64")
    nir = bands[7,:,:].astype("float64")
    ndvi = (nir-red)/(nir+red)
    ndvi = np.expand_dims(ndvi, axis=0) # add a bands axis
    bands = np.concatenate((bands, ndvi), axis=0) # stack the ndvi band
    
    ### HERE WE ARE STACKING THE FIELD ID BAND ###
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        field_ids = src.read().astype("float64")
        field_ids[field_ids == 0] = np.nan
        bands = np.concatenate((bands, field_ids), axis=0)
    
    ### HERE WE ARE STACKING THE CROP TYPE LABELS BAND ###
    labels_path = os.path.join(os.getcwd(), data_path, i, "label/raster_labels.tif")
    with rasterio.open(labels_path) as src:
        bands = np.concatenate((bands, src.read()), axis=0)
    
    # append the multiband raster to the list storing multiband rasters for data in each directory
    stacked_bands.append(bands)

### Reshaping raster data

For each 256 x 256 pixel Sentinel-2 image footprint, we have have created a multiband raster with each band storing reflectance in different wavelengths, computed an NDVI band and appended that to the stack of raster bands, and also stacked bands representing crop type labels for each pixel and a field id indicating which field a pixel belongs to. We have a list of `ndarray` objects of rank 3 with axis 0 for bands, axis 1 for rows, and axis 2 for columns.  

Our goal is to create a tabular dataset where each column represents a variable (e.g. crop type, spectral reflectance, or field id) and each row represents a field. This tabular format is what is required for many machine learning tasks. 

The next stage in our data transformation routine is to reshape the data from an image-style structure (i.e. where variables are stored along the bands or 0 axis) to a tabular style format where variables are stored along the columns axis. A tabular style dataset can be represented as a rank 2 `ndarray` (axis 0 for rows and axis 1 for columns). Therefore, we need a reshaping operation that will transform a rank 3 `ndarray` object to a rank 2 `ndarray` object and arrange variables along the 1 axis. 

NumPy has several tools that can be used for reshaping data. An `ndarray` object has a <a href="https://numpy.org/doc/stable/reference/generated/numpy.reshape.html" target="_blank">`reshape()`</a> method. This method takes in a tuple of integers that describe the shape of the new `ndarray`. For example, let's demonstrate reshaping a `3 x 3` `ndarray` to a `9 x 1` `ndarray`.

In [ ]:
a = np.array([[1, 2, 3], 
             [1, 2, 3],
             [1, 2, 3]])

a_reshaped = a.reshape((9, 1))
print(a_reshaped)

The transpose is another common operation used for reshaping array-like objects. The transpose operation flips the rows and columns of an array. The transpose of a `5 x 4` array is a `4 x 5` array. NumPy `ndarray` objects have a transpose property `T` which returns the transpose of the array. We can demonstrate a few examples of viewing the transpose of an `ndarray` so you can build up an intuition of how it works.

In [ ]:
a = np.array([[1, 2, 3, 4], 
             [1, 2, 3, 4],
             [1, 2, 3, 4]])

a_transposed = a.T
print("Note how the 1 valued elements are now aligned along the columns or 1 axis")
print(a_transposed)

In [ ]:
b = np.array([[3, 3, 3], 
             [4, 4, 4],
             [5, 5, 5]])

b_transposed = b.T
print("Note how the 3 valued elements are now aligned along the rows or 0 axis")
print(b_transposed)

We'll need to reshape the `ndarray` object from rank 3 with shape `(bands, rows, cols)`  to a rank 2 array with shape  `(bands, (rows * cols))`. Instead of arranging the elements for each variable (i.e. reflectance values in different wavelengths, crop type, field id) in an image-style format (like a rank 2 array), the reshape operation will transform the `ndarray` so bands remain on the 0-axis but become rows and the the data values will be arranged along the 1 axis (columns). 

Let's demonstrate this with the first `ndarray` in `stacked_bands`.

In [ ]:
multiband_raster = stacked_bands[0]
print(f"the shape of the first multiband raster in stacked_bands is {multiband_raster.shape}")

rows = multiband_raster.shape[1]
cols = multiband_raster.shape[2]
n_bands = multiband_raster.shape[0]
multiband_reshaped = multiband_raster.reshape(n_bands, rows*cols)
print(f"the shape of the reshaped raster is {multiband_reshaped.shape}")

Now the we have reshaped a multiband raster to a tabular format, but the variables are arranged down the 0 axis (rows) and we want them arranged as columns. We can use a transpose operation to flip our now reshaped rank 2 `ndarray` to the desired tabular format structure. 

In [ ]:
tabular_array = multiband_reshaped.reshape(n_bands, rows*cols).T
print(f"the shape of the transposed array is {tabular_array.shape}")

Now we know how to transform our rank 3 `ndarray` objects representing multiband rasters to an `ndarray` representing a tabular format, we can extend our routine to perform the reshaping operations after the band stacking operations. 

In [ ]:
# stacking bands and then reshaping to tabular format 

# Sentinel-2 band names 
s2_bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

# a list to store numpy ndarray objects representing multiband rasters of Sentinel-2 reflectance data reshaped to tabular format
tables = []

# loop over image directories
for i in image_dirs:
    print(f"stacking Sentinel-2 bands in {i}")
    # a list to store numpy ndarray objects of Sentinel-2 reflectance data for each band
    bands = []
    
    # loop over each band, read in the data from the corresponding GeoTIFF file into an ndarray
    for b in s2_bands:
        band_path = os.path.join(os.getcwd(), data_path, i, b + ".tif")
        with rasterio.open(band_path) as src:
            # append the ndarray storing the Sentinel-2 reflectance data for a band to a list
            bands.append(src.read(1))
    
    # stack all bands in the list to create a multiband raster
    bands = np.stack(bands)
    
    # make NDVI band
    red = bands[3,:,:].astype("float64")
    nir = bands[7,:,:].astype("float64")
    ndvi = (nir-red)/(nir+red)
    ndvi = np.expand_dims(ndvi, axis=0) # add a bands axis
    bands = np.concatenate((bands, ndvi), axis=0) # stack the ndvi band
    
    # add field id band
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        field_ids = src.read().astype("float64")
        field_ids[field_ids == 0] = np.nan
        bands = np.concatenate((bands, field_ids), axis=0)
    
    # add crop type labels band
    labels_path = os.path.join(os.getcwd(), data_path, i, "label/raster_labels.tif")
    with rasterio.open(labels_path) as src:
        bands = np.concatenate((bands, src.read()), axis=0)
    
    # reshape multiband raster to tabular format
    rows = bands.shape[1]
    cols = bands.shape[2]
    n_bands = bands.shape[0]
    reshaped = bands.reshape(n_bands, rows*cols)
    tabular = reshaped.T
    
    # append the tabular array to the list storing tables for the data in each directory
    tables.append(tabular)

## Attribute data operations

### Pandas `DataFrame` and data cleaning

We have applied a series of transformation operations to raster data using NumPy `ndarray` objects. We read several raster datasets stored as GeoTIFF objects into a NumPy `ndarray` object initially creating a multiband raster-like object before reshaping to a tabular-like structure. 

As our data is now in a tabular structure it makes sense to convert it from a NumPy `ndarray` object to Pandas a `DataFrame` object. Pandas `DataFrame` objects, and the Pandas package more generally, are based on NumPy but have been extended and tailored for working with tabular datasets. For example, a NumPy `ndarray` stores data of the same type in an array-like structure (e.g. all elements are integers). A Pandas `DataFrame` can store different type data in different columns (e.g. column 0 is string, column 1 is floating point, etc.), but the values within each column are the same type and each column is typically a `PandasArray` which is based on a NumPy array. Columns in a Pandas `DataFrame` are called `Series` and a `Series` can be objects in your program independent of a `DataFrame`. 

One way of thinking about the links between NumPy and Pandas is: a `Series` is an array-like sequence of values stored in a `PandasArray` which wraps a NumPy `ndarray`,  and a `DataFrame` is creates a tabular structure by combining one or more `Series`. 

Operations on Pandas `DataFrame`s also borrow from NumPy's style such as avoiding for-loops; however, they also provide several features and functions geared towards working with tabular datasets. A selection of these features that are relevant to working with tabular data include:

* Indexing using column names.
* Relational database style operations including key-based joins, conditional filtering and selection of data, and group-by and summarise.
* Support for working with time-series. 
* More tools for handling missing data.

Thus, Pandas `DataFrame` objects are useful for many attribute data transorfmation operations.

To convert a NumPy `ndarray` to a Pandas `DataFrame` we pass the array into the <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame" target="_blank">`DataFrame`'s constructor</a> function helpfully named `DataFrame()`. The constructor function expects the NumPy `ndarray` and a list of column labels as arguments. 


In [ ]:
table_array_0 = tables[0]
print(f"table_array_0 is of {type(table_array_0)} type with shape {table_array_0.shape}")

# convert ndarray to Pandas DataFrame
s2_bands = ['B01', 'B02', 'B03', 'B04','B05', 'B06', 'B07', 'B08','B8A', 'B09', 'B11', 'B12']

# create a DataFrame object
tmp_df = pd.DataFrame(table_array_0, columns=s2_bands + ["ndvi", "field_id", "labels"])
tmp_df

Looking at the `DataFrame` we can clearly see some `NaN` values in the `field_id` column. As each row in this `DataFrame` represents a pixel in the image, rows with `NaN` values in the `field_id` column are pixels which don't have a crop type label. Therefore, we can drop them using the `dropna()` method - this will drop all rows from the `DataFrame` where there is a `NaN` value.

Let's inspect some metadata for the `DataFrame` we have created. 

In [ ]:
tmp_df.info()

The `DataFrame`'s `info()` method returns a summary of the column's data types, count of non-null data, and the memory usage for the object. We can see that the `field_id` and `labels` columns are float64; however, these columns are storing categorical data so integer numbers would be a more appropriate type. Therefore, we can use the `astype()` method to convert these columns to integer type. 

In [ ]:
tmp_df = tmp_df.dropna()
tmp_df = tmp_df.astype({"field_id": "int32", "labels": "int32"})
# Check the cast to integer type has worked. 
# Also, note the reduced memory usage after dropping all the nan rows
tmp_df.info()

### Grouped summaries

Another common attribute operation when working with tabular data is performing grouped aggregations and summaries. For example, often we want to compute the mean, median, min, max, or sum of data values within groups in our dataset. This could be to generate summary tables for reporting purposes or an intermediate step in a data transformation workflow. 

Our goal for this data transformation workflow is to generate average spectral reflectance values in different wavelengths from Sentinel-2 images for each field with a crop type label. So far we have created a table where each row represents one pixel in a 256 x 256 image footprint and we have many pixels per field. We need to compute the average reflectance values for each field. This is a group-by and summarise operation - grouping by field and summarising using the mean. 

A group-by and summarise operation can be conceptualised as a sequence of split-apply-combine steps <a href="https://wesmckinney.com/book/data-aggregation.html#groupby_fundamentals" target="_blank">McKinney (2022)</a>:

* **Split** your dataset into groups.
* **Apply** a function to values in each group as a summary.
* **Combine** the results of applying the function to each group. 

For our dataset we need to group-by `field_id` and `labels` (crop type column) and compute the mean of spectral reflectance values within each group. 

Pandas `DataFrame`s have a `groupby()` method that can take in a list of one or more column names. Calling this method returns a `GroupBy` object that creates groups from your dataset for each of the unique values of the grouping columns and can be used to apply summary operations to each group. 

In [ ]:
# create a group using field id and crop type
tmp_df_groups = tmp_df.groupby(["field_id", "labels"])
print(tmp_df_groups)

Finally, we need to apply our summary operations to each group. We can do this by calling a function on the `GroupBy` object. A useful function for data exploration tasks is calling `size()` which tells us the number of observations in each group. 

In [ ]:
# size of groups
tmp_df_groups.size()

By calling `size()` on the `GroupBy` object we can see that we have one group with a `field_id` of `2595` and a crop type label of `1` (wheat). There are 15 observations in this group. If we want to compute the mean of the spectral reflectance values within each group we can call `mean()` instead of `size()`. 

In [ ]:
# mean spectral reflectance values per group
tmp_df_groups.mean()

We're now in a position to update our data transformation routine to include converting the `ndarray` objects in a tabular-like structure to Pandas `DataFrames`, data cleaning to drop `NaN` pixels, and computing the mean spectral reflectance values for each `field_id` and crop type `label` combination. 

In [ ]:
# stacking bands, reshaping to tabular format, convert to DataFrame, and summarise by field id and crop type 

# Sentinel-2 band names 
s2_bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

# a list to store DataFrames 
dfs = []

# loop over image directories
for i in image_dirs:
    print(f"stacking Sentinel-2 bands in {i}")
    # a list to store numpy ndarray objects of Sentinel-2 reflectance data for each band
    bands = []
    
    # loop over each band, read in the data from the corresponding GeoTIFF file into an ndarray
    for b in s2_bands:
        band_path = os.path.join(os.getcwd(), data_path, i, b + ".tif")
        with rasterio.open(band_path) as src:
            # append the ndarray storing the Sentinel-2 reflectance data for a band to a list
            bands.append(src.read(1))
    
    # stack all bands in the list to create a multiband raster
    bands = np.stack(bands)
    
    # make NDVI band
    red = bands[3,:,:].astype("float64")
    nir = bands[7,:,:].astype("float64")
    ndvi = (nir-red)/(nir+red)
    ndvi = np.expand_dims(ndvi, axis=0) # add a bands axis
    bands = np.concatenate((bands, ndvi), axis=0) # stack the ndvi band
    
    # add field id band
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        field_ids = src.read().astype("float64")
        field_ids[field_ids == 0] = np.nan
        bands = np.concatenate((bands, field_ids), axis=0)
    
    # add crop type labels band
    labels_path = os.path.join(os.getcwd(), data_path, i, "label/raster_labels.tif")
    with rasterio.open(labels_path) as src:
        bands = np.concatenate((bands, src.read()), axis=0)
    
    # reshape multiband raster to tabular format
    rows = bands.shape[1]
    cols = bands.shape[2]
    n_bands = bands.shape[0]
    reshaped = bands.reshape(n_bands, rows*cols)
    tabular = reshaped.T
    
    ### HERE WE CONVERT TO DATAFRAMES AND DROP NAN VALUES ###
    tmp_df = pd.DataFrame(tabular, columns=s2_bands + ["ndvi", "field_id", "labels"])
    tmp_df = tmp_df.dropna()
    
    # append DataFrame to list of DataFrames 
    dfs.append(tmp_df)

### HERE WE COMBINE ALL THE DATAFRAMES FOR EACH 256 x 256 IMAGE FOOTPRINT AND COMPUTE THE MEAN VALUES PER GROUP ###
dfs = pd.concat(dfs)
dfs = dfs.astype({"field_id": "int32", "labels": "int32"})
dfs = dfs.groupby(["field_id", "labels"]).mean().reset_index()

Let's quickly inspect the output. We should have one row per `field_id` and crop type `label` group. The `DataFrame` storing the results of this routine are referenced by the variable `dfs`.

In [ ]:
dfs

## Raster-vector operations and vector operations

We're almost at the stage where we've processed a number of GeoTIFF files stored across many directories into a tabular dataset in a `DataFrame` object ready for machine learning. However, there are two more columns we need to create and append to the `DataFrame`. The first is a `geometry` column recording the centroid of each field. This allows us to keep a record of each field's geographic location. We'll also use this centroid to identify the district (an administrative boundary below the State-level in India) each field is located in. We'll use this geographic information to ensure training and test data are not spatially correlated in the machine learning workflows (but, that is a discussion for later). 

To compute the centroid for each field we need to perform some raster-vector operations where each raster dataset is converted to a vector dataset. This is called vectorisation and can be achieved using `rasterio`'s <a href="https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html#rasterio.features.shapes" target="_blank">`shapes()`</a> function which returns the shape and value of connected regions in a raster dataset. Pixels belonging the same field in a raster layer should be connected (i.e. their edges touch) and they should have the same value (field id). Thus, applying the `shapes()` function to the raster layer of field ids should return vector polygons for each field outline.

To do this we'll need to use the `field_ids.tif` files. Let's quickly inspect these files again.

In [ ]:
for i in image_dirs:
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        print(f"Printing metadata for field_ids.tif in {i}")
        pprint.pprint(src.meta)
        print("")

We have printed dictionary objects of the metadata for each of the `field_ids.tif` files. You will notice that not all files have the same coordinate reference system (the `crs` slot; CRS). This will be a problem if we try and create a `geometry` column where each row stores the point centroid for each field. We'll need all the points in our column to be in the same coordinate reference system. So, we'll need to add a step to this process that reprojects all the point centroids to a common CRS. Here, we'll reproject to latitude and longitude or `EPSG:4326`. 

Let's look at what the `shapes()` function returns for the first `field_ids.tif` file in `image_dirs` (remember `image_dirs` is a list of directories storing GeoTIFF files). 

The `shapes()` function takes in a NumPy `ndarray` of raster values (generated by `src.read()` which reads the raster values from the GeoTIFF file into a NumPy `ndarray` in memory) and can be converted to a list object. The list object stores a tuple for each shape in the raster data. The first element of the tuple is a dictionary object of coordinates and the type of geometry (e.g. point, line, polygon). The second element of the tuple is the value that corresponds to the geometry. 

We can see below that we have two elements in our list. The first has a value of 2592 which corresponds to the field id in the raster layer. 

In [ ]:
for i in image_dirs[0:1]:
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        # shapes is a generator
        shapes = features.shapes(src.read(), transform=src.transform)
        
        # list of geometry and shape value
        field_shapes = list(shapes)
        
        pprint.pprint(field_shapes)

At this stage, we've converted our raster data to a list of numbers representing coordinates for the shape. We now need to turn this list of coordinates into a geometry object. In Python, geometry objects as <a href="https://shapely.readthedocs.io/en/stable/geometry.html" target="_blank">Shapely</a> `Geometry` objects. The `geometry` column in a GeoPandas `GeoDataFrame` is a `Series` of Shapely `Geometry` objects. 

In [ ]:
for i in image_dirs[0:1]:
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        # shapes is a generator
        shapes = features.shapes(src.read(), transform=src.transform)
        
        # list of geometry and shape value
        field_shapes = list(shapes)
        
        # create a list of Shapely Geometry objects
        geom = []
        for s in field_shapes:
            geom.append(shapely.geometry.shape(s[0]))
        
        print(geom)

Next, we compute the centroid for the polygon shape of the field. Computing a centroid is a geometry operation where the shape's geometry is converted from a polygon to a point feature. To efficiently compute the centroid for the shapes returned by `shapes()` we can convert the list of `Geometry` objects to a `GeoSeries` and then use the `GeoSeries` `centroid` attribute to return a `GeoSeries` of polygon centroids. 

Inspecting this `GeoSeries` should reveal a sequence of point `Geometry` objects have been computed.

In [ ]:
for i in image_dirs[0:1]:
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        # shapes is a generator
        shapes = features.shapes(src.read(), transform=src.transform)
        
        # list of geometry and shape value
        field_shapes = list(shapes)
        
        # create a list of Shapely Geometry objects
        geom = []
        for s in field_shapes:
            geom.append(shapely.geometry.shape(s[0]))
        
        # compute centroids
        geom = gpd.GeoSeries(geom, crs=src.crs).centroid
        
        print(geom)

Now we have computed the centroid for each field, we can convert the points to a common coordinate reference system (`EPSG:4326`) using GeoPandas `to_crs()` method. We'll also create another `Series` of field ids and combine the `Series` of field ids with the `GeoSeries` of field centroids to create a `GeoDataFrame`. We'll also drop shapes with the value of 0 as these shapes don't have a crop type label. 

In [ ]:
for i in image_dirs[0:1]:
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        # shapes is a generator
        shapes = features.shapes(src.read(), transform=src.transform)
        
        # list of geometry and shape value
        field_shapes = list(shapes)
        
        # create a list of Shapely Geometry objects
        geom = []
        for s in field_shapes:
            geom.append(shapely.geometry.shape(s[0]))
        
        # compute centroids
        geom = gpd.GeoSeries(geom, crs=src.crs).centroid
        
        # reproject to EPSG 4326
        geom = geom.to_crs("EPSG:4326")
        
        # create a Series of field ids
        field_ids = []
        for f in field_shapes:
            field_ids.append(f[1])

        field_ids = pd.Series(field_ids).astype("int32")
        
        # Combine the Series and GeoSeries into a DataFrame
        field_id_df = gpd.GeoDataFrame({'field_id': field_ids, 'geometry': geom})
        
        # drop shapes with value 0
        field_id_df = field_id_df.loc[field_id_df["field_id"] > 0, :]
        
        print(field_id_df)

Now, we're able to create a small routine that will loop over our list of directories and create a `GeoDataFrame` with a column for the field id and a column for the field centroid. 

In [ ]:
field_id_dfs = []

for i in image_dirs:
    field_id_path = os.path.join(os.getcwd(), data_path, i, "field/field_ids.tif")
    with rasterio.open(field_id_path) as src:
        # shapes is a generator
        shapes = features.shapes(src.read(), transform=src.transform)
        
        # list of geometry and shape value
        field_shapes = list(shapes)
        
        # create a list of Shapely Geometry objects
        geom = []
        for s in field_shapes:
            geom.append(shapely.geometry.shape(s[0]))
        
        # compute centroids
        geom = gpd.GeoSeries(geom, crs=src.crs).centroid
        
        # reproject to EPSG 4326
        geom = geom.to_crs("EPSG:4326")
        
        # create a Series of field ids
        field_ids = []
        for f in field_shapes:
            field_ids.append(f[1])

        field_ids = pd.Series(field_ids).astype("int32")
        
        # Combine the Series and GeoSeries into a DataFrame
        field_id_df = gpd.GeoDataFrame({'field_id': field_ids, 'geometry': geom})
        
        # drop shapes with value 0
        field_id_df = field_id_df.loc[field_id_df["field_id"] > 0, :]
        
        field_id_dfs.append(field_id_df)

field_id_dfs = pd.concat(field_id_dfs)
field_id_dfs

## Joins

### Key-based joins

We now have two separate data objects in our Python program. We have a `DataFrame` storing average spectral reflectance values for each field, field id, and crop type label attributes. We also have a `GeoDataFrame` storing the field id attribute and the field centroid as a point `Geometry`. 

When two tables have a matching column(s) we can use join operations to combine them. Rows in both tables are matched using common values in the matching column(s) and the joined table has columns from both tables. 

Joining tables is a common operation in relational databases using SQL and the same operations can be implemented in Pandas using <a href="https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging" target="_blank">`merge()`</a> functions. 

Some important concepts for join operations:

* The columns with values used to match rows are called often called **keys**.
* **one-to-one** joins are where there is exactly one match between rows in the two tables being joined.
* **many-to-one** joins are where a row in one table can match one or more rows in another table.
* **left joins** keep all rows in the left table and only matching rows in the right table. 
* **inner joins** keep only matching rows in the left and right tables. 


The Pandas <a href="https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging" target="_blank">`merge()`</a> docs and <a href="https://wesmckinney.com/book/data-wrangling.html#prep_merge_join" target="_blank">McKinney (2022)</a> provide useful explanations for how join operations work.

Let's use consider these concepts in the context of joining our `DataFrame` `dfs` storing average spectral reflectance values and crop type labels and our `GeoDataFrame` `field_id_dfs` which stores the field centroids. 

The matching column in both tables is `field_id`. This the joining key. 

We are joining the two tables on `field_id` which should be unique to each field. Therefore, we are implementing a one-to-one join. 

As we're using the field centroids for subsequent operations, we only want to keep fields that have a centroid value. Therefore, we'll use an inner join.

Pandas `merge()` function expects the following arguments:

* `left` - left table in the join.
* `right` - right table in the join.
* `how` - whether to use a left or inner join.
* `left_on` - columns in left table to use as keys.
* `right_on` - columns in the right table to use as keys.

Let's join the two tables and inspect the result. If the join is successful you should see a `geometry` column appended to the columns in `dfs`.

In [ ]:
joined_df = pd.merge(left=dfs, right=field_id_dfs, how="inner", left_on=["field_id"], right_on=["field_id"])
# display on the first few rows
joined_df.head()

In [ ]:
# convert joined_df to GeoDataFrame
joined_df = gpd.GeoDataFrame(joined_df, geometry=joined_df.geometry, crs="EPSG:4326")
type(joined_df)

### Spatial Joins

Spatial join operations join the attributes of two vector layers based on their relationship in space. For example, if we have a `GeoDataFrame` storing field boundaries (polygon geometries) and field attributes and another `GeoDataFrame` storing shire boundaries (polygon geometries) and a shire name as an attribute, we can join the the two tables based on the largest intersection (overlap) between field boundaries and shire boundaries. If the field boundaries `GeoDataFrame` was the left table in the spatial join, for each row (or geometry feature) the shire name from the shire with largest intersection would be joined to that table in a new column. 

GeoPandas provides an `sjoin()` function that can be used for spatial joins of two `GeoDataFrames`. The `sjoin()` function expects the following as arguments:

* `left_df` - left `GeoDataFrame` in the spatial join.
* `right_df` - right `GeoDataFrame` in the spatial join - columns from the `right_df` will be joined to `left_df`.
* `how` - whether to use a left, inner, or right join.
* `predicate` - a binary predicate that defines the spatial relationship between features in `right_df` and `left_df`. 

Binary predicates that can be used are:

* intersects
* contains
* crosses
* within
* touches
* overlaps

Intersects is the default predicate for spatial joins in GeoPandas. 

To complete our data transformation routine we need to add a column to `joined_df` that stores the District that the field is located in. We can do this using a spatial join based on the intersect of the field's centroid (point geometry) and the shape of the District (polygon geometry). 

But, we need to read in District geometries for India obtained from <a href="https://www.geoboundaries.org" target="_blank">geoBoundaries</a>. 

In [ ]:
india_districts = gpd.read_file(os.path.join(os.getcwd(), "week-4", "india-adm", "geoBoundaries-IND-ADM2_simplified.topojson"))
india_districts.head()

Let's quickly tidy up the India Districts `GeoDataFrame` to keep only the District name and `geometry` columns.

In [ ]:
india_districts = india_districts.loc[:, ["shapeName", "geometry"]]
india_districts.columns = ["district", "geometry"]
india_districts = india_districts.set_crs("EPSG:4326")
india_districts.head()

In [ ]:
india_districts.plot(column="district")

That looks like India. Let's implement our final data transformation step and perform a spatial join to add a District column to `joined_df`.

In [ ]:
# spatial join
joined_df_district = gpd.sjoin(
    left_df=joined_df, 
    right_df=india_districts, 
    how="inner", 
    predicate="intersects"
)
joined_df_district.head()

### Save file

Finally, let's write our processed data ready for training and testing a machine learning model to file.

In [ ]:
# save file
out_path = os.path.join(os.getcwd(), "week-4", "processed_data.geojson")
joined_df_district.to_file(out_path)